<a href="https://colab.research.google.com/github/aryanfarswan77/prasunet/blob/main/image_and_fooditems_recognisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install tensorflow keras kaggle


In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os


In [ ]:
# Use Kaggle API to download the dataset or upload it manually
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/  # Ensure you upload your 'kaggle.json' API key before running this
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d dansbecker/food-101

# Unzip the downloaded dataset
!unzip food-101.zip


In [ ]:
# Image Data Generator for preprocessing the dataset
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Normalize images

# Training data generator
train_generator = datagen.flow_from_directory(
    'food-101/images',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Validation data generator
validation_generator = datagen.flow_from_directory(
    'food-101/images',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
# Load the ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the pre-trained layers

# Define the full model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Output layer for classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model structure
model.summary()


In [ ]:
# Train the model with training data and validate on the validation set
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples // 32,
    validation_steps=validation_generator.samples // 32
)


In [ ]:
# Example mapping of food classes to their calorie information (you'll need to extend this)
food_calories = {
    'apple_pie': 237,
    'baby_back_ribs': 290,
    'baklava': 220,
    # Add entries for all 101 classes from the dataset
}


In [ ]:
# Define a function to predict food class and estimate calories
def predict_and_estimate_calories(image_path, model, food_calories):
    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict the food class
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction[0])

    # Get the class label
    predicted_label = list(train_generator.class_indices.keys())[predicted_class]

    # Estimate calories
    estimated_calories = food_calories.get(predicted_label, "Calorie information not available")

    return predicted_label, estimated_calories

# Example Usage
image_path = 'path_to_some_test_image.jpg'  # Replace with the path to your test image
predicted_food, calories = predict_and_estimate_calories(image_path, model, food_calories)
print(f"Predicted Food: {predicted_food}, Estimated Calories: {calories}")
